# Linear regression for an actual data set

We consider linear regression for an actual data set. We will use simple features of pandas (see https://pandas.pydata.org/) for data analysis.

**There are 9 questions to answer.**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
%matplotlib inline

# set up the random number generator: given seed for reproducibility, None otherwise
# (see https://numpy.org/doc/stable/reference/random/generator.html#numpy.random.default_rng)
my_seed = 1
rng = np.random.default_rng(seed=my_seed) 

# Dataset 

We use the "day.csv" dataset from https://archive-beta.ics.uci.edu/dataset/275/bike+sharing+dataset (which we renamed when downloading). This dataset records daily counts of bike rentals, together with calendar information and weather conditions. In particular, the field *cnt* gives the total number of rentals that day. This is the quantity we want to predict by linear regression. 

In [ ]:
df = pd.read_csv('BikeSharingDataset.csv')
df.head()
df.info() # if only data types are wanted: print(df.dtypes)

### Cleaning-up and constructing the data set

We remove the first two columns (index of element in the dataset and exact date), as well as the distinction between casual and registered users as the corresponding features are not of interest to us. See https://www.kaggle.com/code/gauravduttakiit/bike-sharing-multiple-linear-regression for further checks and manipulations to ensure that the data is correct (checking for missing data, duplicates, etc).

In [ ]:
# use the drop() function in panda, see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
# argument 'inplace=True' to permanently remove the column
df.drop(['instant','dteday','casual','registered'],axis=1,inplace=True)
df.head()
print(df.shape)

For each rental day, the following features are provided:
 - season: 1 = spring, 2 = summer, 3 = fall, 4 = winter
 - yr: year, here 0 for 2011 and 1 for 2012
 - mnth: month, numbered from 1 to 12
 - holiday: 0 if no holiday, 1 if there is a holiday (extracted from http://dchr.dc.gov/page/holiday-schedule)
 - weekday: 0 (Sunday) to 6 (Saturday)
 - workingday: 0 for non-working day, 1 for working day 
 - weathersit: weather favorability rating from 1 (clear day) to 4 (rain, fog)
 - temp: normalized temperature 
 - atemp: normalized temperature feels like
 - hum: normalized humidity 
 - windspeed: normalized wind speed
 - cnt: the number of rented bicycles (target attribute to be predicted)

So, we have features which are real numbers (*temp, atemp, hum,	windspeed*), binary (*holiday, workingday*), and ordinal features (eg. *season, weekday, weathersit*). We can consider all of them as real numbers features. To get some idea of the data values, we can use the panda function *describe()*, which produces some descriptive statistics (see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.describe.html).

In [ ]:
df.describe()

Note that the *yr* data is more or less evenly distributed (as the mean is close to 0.5). On the other hand, there are only a handful of holidays (see the small mean value of *holiday*), so this variable should probably not be considered.

We next randomly split the data into train and test sets, before conducting any analysis, as such an analysis will be based on the train set only. This is done using a dedicated scikit-learn function (see https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html).

In [ ]:
fraction_train = 0.7 
fraction_test = 1.0 - fraction_train
df_train, df_test = train_test_split(df, train_size = fraction_train, test_size = fraction_test)
df_train.info()
df_test.info()

### Exploring the data

We start by exploring the dataset to understand how the target attribute (*cnt*) depends on each of the features. We start by using the plot function of *pandas* (see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html). It would be better to do this on the training data only, but we do it on the full data here.

In [ ]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 10))
# df_train.columns[:-1] is the list of all columns, except the last one (the target attribute 'cnt')
# this can be enumerated with an index using https://docs.python.org/3/library/functions.html#enumerate
# position in the axes: vector [i,j] with i is the line number, and j the column index
for idx, feature in enumerate(df_train.columns[:-1]):
    df.plot(feature, "cnt", subplots=True,  kind="scatter", ax=axes[idx // 4, idx % 4]) 
plt.show()

It is not so easy to have some quantitative impression from scatter plots of categorical variables. Boxplots can be more appropriate.

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(2,3,1)
sns.boxplot(x = 'season', y = 'cnt', data = df_train)
plt.subplot(2,3,2)
sns.boxplot(x = 'mnth', y = 'cnt', data = df_train)
plt.subplot(2,3,3)
sns.boxplot(x = 'weathersit', y = 'cnt', data = df_train)
plt.subplot(2,3,4)
sns.boxplot(x = 'holiday', y = 'cnt', data = df_train)
plt.subplot(2,3,5)
sns.boxplot(x = 'weekday', y = 'cnt', data = df_train)
plt.subplot(2,3,6)
sns.boxplot(x = 'workingday', y = 'cnt', data = df_train)
plt.show()

**Question 1.** Based on these plots, describe features which have an influence on the target attribute.

The plots suggest that there is a linear dependence between the target and some features such *temp* or *atemp*. There are also noticeable correlations with *mnth* and *season*. There seems to be also more rentals on working days, and when the weather is fine.

### Data normalization

The sizes of the data entries are on rather different scales, as made precise through the output of the *describe()* function above. We therefore renormalize the data by rescaling the features, so that all entries are between 0 and 1 (see https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html).

In [ ]:
scaler = MinMaxScaler()
df_train[:] = scaler.fit_transform(df_train[:])
df_train.describe()

The minimum and maximum are indeed 0 and 1 for all variables.

### Features and target correlations

We now estimate more quantitatively the level of linear dependence between the features and the target variable. A good measure of the linear relationship between two vectors is the [Pearson correlation](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) (see also Chapter 25 [Shalev-Shwartz] for a discussion on how it relates to feature selection). 

Correlations can also be computed with the *pandas* library, using the dataframe method *corr* (see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html), and plotted with the heatmap function of the statistical data visualization library *seaborn* (see https://seaborn.pydata.org/generated/seaborn.heatmap.html)/ 

In [ ]:
plt.figure(figsize = (15,10))
# sns.set_theme()
# other 'Diverging colormaps' can be used, see https://matplotlib.org/stable/tutorials/colors/colormaps.html
sns.heatmap(df_train.corr(method='pearson'), annot = True, linewidths=.5, cmap="coolwarm")
plt.show()

We next break down the correlations into correlations between the variables we are looking for to predict the result, and correlations between these variables and the target attribute *cnt*. The latter correlations can be computed with the function *corrwith* (see https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corrwith.html), which needs as argument another dataframe as an argument to compute the pairwise correlations with. 

In [ ]:
# df_train_variables is df_train without the column of the target attribute 'cnt'
# alternative command: use df[df.columns.difference(['cnt'])]
df_train_variables = df_train[df_train.columns[:-1]]
# get features correlation coefficients
df_train_variables.corr(method='pearson')

Some quantities are strongly correlated (in particular *temp* and *atemp*). This will require some care when choosing the features for prediction.

We next compute correlations with the target attribute. 

In [ ]:
df_train_variables.corrwith(df_train['cnt'], axis=0, method='pearson')

**Question 2.** Compare these numbers with the corresponding ones in the heatmap above.

We recover the numbers displayed in the heatmap above.

The sample above shows features correlations with the target. Some of them (in fact, all of of them) are not *null*. This means that it makes sense to try to predict the output using a linear method (although in the end it may not be sufficient).

# Training with all features (despite strong correlations)

Some attributes in our dataset have strong correlations (one speaks of *multicollinearity*, see https://en.wikipedia.org/wiki/Multicollinearity), which can lead to issues in solving the regression problem, as the design matrix may be ill-conditioned. We start by a naive estimation where we pretend that we have not noticed the strong correlations, in order to highlight the kind of problems that arise.

We start by decomposing the train data into the features $X$ and the target attribute $y$.

In [ ]:
# from sklearn.utils import shuffle
# df_shuffled = shuffle(df, random_state=123)
df_train, df_test = train_test_split(df, train_size = fraction_train, test_size = fraction_test)
df_train[:] = scaler.fit_transform(df_train[:])
X = df_train[df_train.columns[:-1]]
y = df_train["cnt"]
df_test[:] = scaler.transform(df_test[:]) 
X_test = df_test[df_test.columns[:-1]]
y_test = df_test["cnt"]
print(X.head())
print('\n------------------------------------\n')
print(y.head())

**Question 3.** Check that the test data has been properly normalized. Are the minima and maxima of all features 0 and 1, respectively? If not, why? 

In [ ]:
df_test.describe()

We use the rescaling of the train data. It is therefore possible that some features in the test do not span all the values between 0 and 1 (for instance, the minimum can be below 0; or the maximum below 1).

### Plain training of a linear model

We first do some training on the whole data set, with all features kept.

**Question 4.** Train a linear regression model on the training dataset and look at the feature weights. The acquired weights are in the *coef_* parameter of a regressor class. It is good to re-run the estimation several times by randomly re-splitting the data set into train and test sets. What do you observe?

In [ ]:
from sklearn import linear_model
# build a model
linear_regressor = linear_model.LinearRegression() 
# train the model using all data
linear_regressor.fit(X, y) 
# output coefficients
print('\nCoefficients and their weights:')
# zip function to aggregate https://docs.python.org/3.3/library/functions.html#zip
# "{:.3f}".format(number) to display number with 3 decimal places
# or alternatively round(number,3), as used below
for pair in zip(df_train.columns, linear_regressor.coef_):
  print(pair[0],' -> ',"{:.3f}".format(pair[1])) 

We see that the weights for linearly dependent features are significantly greater in absolute values than for other features. More importantly, when re-running the fitting procedure with newly randomly split train/test sets, the weights for these features change dramatically (look for instance at *temp* and *atemp*; although, interestingly, the sums of these weights is more or less constant) -- as opposed to other weights, such as *yr*, which are quite stable. This is related to the fact that the matrix $X^T X$ is ill-conditioned when two features are too strongly correlated, so that the linear system $X^T X w = X^T y$ defining $w$ is difficult to solve, in the sense that small variations in $y$ can lead to strong variations in $w$. 

For the remainder of this notebook, we remove the *temp* field, and only keep *atemp*.

In [ ]:
df.drop('temp',axis=1,inplace=True)

# Training with some regularization 
We next consider training with some regularization in order to obtain the best bias/variance trade-off. The norm of weights multiplied by the regularization coefficient $\alpha$ is added to the cost function to minimize. We consider both lasso ($\ell^1$ norm) and ridge ($\ell^2$ norm) regression. We train the associated regularized cost functions with their defaults parameters, see https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html and https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html

In [ ]:
from sklearn.linear_model import Lasso, Ridge

### One shot training

We can run the optimization with lasso and ridge regression various times, by randomly reshuffling the data set in order to get some intuition on the variability of the coefficients. We work here only on the training data, and do not test yet the quality of the prediction on the test set.

**Question 5.** Change the value of the regularization parameter $\alpha$ in order to see the impact on the fitted coefficients. What do you observe?

In [ ]:
lasso = Lasso(alpha=0.01)
df_train, df_test = train_test_split(df, train_size = fraction_train, test_size = fraction_test)
df_train[:] = scaler.fit_transform(df_train[:])
X = df_train[df_train.columns[:-1]]
y = df_train["cnt"]
lasso.fit(X, y)
print('Intercept -> ', round(lasso.intercept_,3))
for pair in zip(df_train.columns, lasso.coef_):
  print(pair[0],' -> ',"{:.3f}".format(pair[1])) 

For large values of $\alpha$, many coefficients are 0, and the non zero coefficients are rather stable from one realization of the test/train split to the other. For small values of $\alpha$, almost all coefficients are non zero, and can substantially change from one realization to another.

**Question 6.** Do the same for ridge regression.

In [ ]:
ridge = Ridge(alpha=0.0001)
ridge.fit(X, y)
print('Intercept -> ', round(ridge.intercept_,3))
for pair in zip(df_train.columns, ridge.coef_):
  print(pair[0],' -> ',"{:.3f}".format(pair[1])) 

The coefficients for ridge regression and lasso are quite similar for small values of the regularization parameter. A nice aspect of lasso regression though is that it puts to 0 uninformative features, whereas ridge regression still assigns them a non zero (although small) coefficient.

### Training paths

We now explore more systematically the dependence of the result as a function of the regularization coefficient. The aim is to find the best value of $\alpha$ for prediction, using the test set. Before doing this, we look more closely at how the coefficients of the fit change as $\alpha$ is increased.

For each coefficient value from the vector of values *alphas_lasso* for the parameter $\alpha$, we train the Lasso regressor and write the weights into the corresponding rows of the *coefs_lasso* matrix. The same is then done for ridge regression. This will allow us to look at how coefficients vary as $\alpha$ is changed. Note that it would be possible instead to plot results as a function of the magnitude of the coefficients, using the scikit-learn function https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.lars_path.html

In [ ]:
# we use coefficients logarithmically spaced in order to cover a wide range of values 
alphas_lasso = np.logspace(-5, 1, 100)
coefs_lasso = np.zeros((alphas_lasso.shape[0], X.shape[1])) 

for i, al in enumerate(alphas_lasso):
    lasso = Lasso(alpha = al)
    lasso.fit(X, y)
    for j, coef in enumerate(lasso.coef_):
        coefs_lasso[i][j] = coef
        
plt.figure(figsize=(10, 8))
for coef, feature in zip(coefs_lasso.T, df.columns):
    plt.semilogx(alphas_lasso, coef, label=feature)
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("Alpha")
plt.ylabel("Feature weight")
plt.title("Lasso regression")
plt.show()

We next do the same for ridge regression.

In [ ]:
alphas_ridge = np.logspace(-5, 4, 100)
coefs_ridge = np.zeros((alphas_ridge.shape[0], X.shape[1]))

for i, al in enumerate(alphas_ridge):
    ridge = Ridge(alpha = al)
    ridge.fit(X, y)
    for j, coef in enumerate(ridge.coef_):
        coefs_ridge[i][j] = coef
        
plt.figure(figsize=(10, 8))
for coef, feature in zip(coefs_ridge.T, df.columns):
    plt.semilogx(alphas_ridge, coef, label=feature)
plt.legend(loc="upper right", bbox_to_anchor=(1.4, 0.95))
plt.xlabel("Alpha")
plt.ylabel("Feature weight")
plt.title("Ridge regression")
plt.show()

For illustration, we can also compute the regression path as a function of the magnitude of the coefficients, for lasso.

In [ ]:
# dataframes need to be converted to numpy format
XX = X.to_numpy()
yy = y.to_numpy()
_, _, coefs = linear_model.lars_path(XX, yy, method="lasso")

# plot the corresponding result
tau = np.sum(np.abs(coefs.T), axis=1)
plt.figure(figsize=(10, 8))
plt.xlabel("L1 norm of coefficients")
plt.plot(tau, coefs.T, marker="o")
plt.legend(list(df.columns), bbox_to_anchor=(1.4, 0.95))
plt.show()

**Question 7.** What are the largest coefficients? Which coefficients are the most important? (in the sense that they are non zero first when performing lasso)

Note that, in all these plots, the coefficient for the year is large and positive, and is the first one to be non zero when decreasing the penalty coefficient (starting from a large value). This is because there were more rentals in average in the second year (as we check below; see the last column 'cnt' in the reported statistics). The average of the other features are rather similar. This could be due to the fact that the bike rental system was better known the second year, and therefore attracted more customers, irrespectively of weather conditions, etc. 

The coefficient related to the working day is the first one to be non zero when looking at the LARS path, but it eventually converges to a rather small value.

Moreover, the coefficient corresponding to temperature is large, and, to a lesser extent, the one related to the season.

In [ ]:
df_0 = df[df["yr"] == 0]
df_1 = df[df["yr"] == 1]
print('Statistics for the first year \n')
print(df_0.describe())
print('\n ------------------------------ \n')
print('Statistics for the second year \n')
print(df_1.describe())

### Cross validation

We next perform cross validation in order to choose the best coefficient. The metric for the quality of the results is the mean square error (MSE) computed on the test set. We explore two approaches: randomly splitting the full data set into test/train sets, or using k-fold cross validation. For the latter method, we use built-in routines in scikit-lean to divide the sample into $k$ blocks, each of them being considered as a test set when training is performed on the remaining $k-1$ blocks. The MSE for a given value of $\alpha$ is averaged over the test MSEs over the folds. 

We start by computing the MSE and the regression score for a given value of $\alpha$ in a pedestrian way, in order to get a feeling for what is being done. Everything is done for Lasso, but the adaptation to ridge regression would be straightforward.

**Question 8.** Complete the code below: in one case, you need to compute 'by hand' something computed by a built-in scikit-learn function, and in the other case you need on the contrary to use a built-in scikit-learn function.

In [ ]:
al = 0.0001

df_train, df_test = train_test_split(df, train_size = fraction_train, test_size = fraction_test)
df_train[:] = scaler.fit_transform(df_train[:])
X = df_train[df_train.columns[:-1]]
y = df_train["cnt"]
df_test[:] = scaler.transform(df_test[:])
X_test = df_test[df_test.columns[:-1]]
y_test = df_test["cnt"]

lasso = Lasso(alpha = al)
lasso.fit(X, y)
# report coefficients
print('Intercept:', lasso.intercept_)
print( 'Rounded coef:', list(map(lambda c: round(c, 3) , lasso.coef_) )) 
# see function score() on https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html
print('\nR2  (train) = ',lasso.score(X,y))
y_pred = lasso.predict(X)
E2_test = ((y-y_pred)**2).sum() 
print('R2  (train) = ',1-E2_test/sum( (y-np.mean(y))**2 ),' computed by hand')
# we next compute the MSE equivalently 
from sklearn.metrics import mean_squared_error
print('MSE (train) = ',mean_squared_error(lasso.predict(X), y))
print('MSE (train) = ',((y-lasso.predict(X))**2).mean(),' computed by hand\n')
# compute the prediction error on the test set
# one can check that this is equivalent to lasso.intercept_+sum(lasso.coef_*X[i]) for each index i  
y_pred = lasso.predict(X_test)
print('R2   (test) = ',lasso.score(X_test,y_test))
print('MSE  (test) = ',((y_test-y_pred)**2).mean())
# plotting the results
plt.title('Correlation between actual and predicted values')
plt.xlabel('actual values')
plt.ylabel('predicted values')
plt.scatter(y_test,y_pred,color='royalblue')
plt.plot([0,1],[0,1],color='red')
plt.show()

We now more precisely quantify the variability of the outputs by performing several realizations.

In [ ]:
al = 0.0001
nb_real = 100

R2_test = np.zeros(nb_real) 
MSE_test = np.zeros(nb_real)

lasso = Lasso(alpha = al)
for real in range(nb_real):
  df_train, df_test = train_test_split(df, train_size = fraction_train, test_size = fraction_test)
  df_train[:] = scaler.fit_transform(df_train[:])
  X_train = df_train[df_train.columns[:-1]]
  y_train = df_train["cnt"]
  lasso.fit(X, y)
  df_test[:] = scaler.transform(df_test[:])
  X_test = df_test[df_test.columns[:-1]]
  y_test = df_test["cnt"]
  R2_test[real] = lasso.score(X_test,y_test)
  # equivalently one could use a built-in function: 
  # import function as --> from sklearn.metrics import mean_squared_error 
  # call function as --> mean_squared_error(y_pred, y)
  # with y_pred = lasso.predict(X)
  MSE_test[real] = ((y_test-lasso.predict(X_test))**2).mean() 
    
print('Average R2  (test) =', R2_test.mean())
print('Average MSE (test) =', MSE_test.mean())

In practice, one rather uses a single data set, and relies on built-in cross validation function such as [__LassoCV__](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LassoCV.html). It takes as input a list of values of $\alpha$ and calculates the MSE for each of them. After training, the regressor will contain the variable __mse\_path\___, a matrix of size `len(alpha) x n_folds`, where `n_folds` (the number of blocks in cross-validation), containing the MSE values on the test for the corresponding runs. In addition, the variable __alpha\___ will store the selected (optimal) value of the regularization parameter, and in __coef\___, will have the trained weights corresponding to this __alpha___.

Note that the regressor can change the order in which it iterates through the alphas values. In order to map it to the MSE matrix, it is better to use the regressor variable __alphas___.

In [ ]:
from sklearn.linear_model import LassoCV

In [ ]:
df_train, df_test = train_test_split(df, train_size = fraction_train, test_size = fraction_test)
df_train[:] = scaler.fit_transform(df_train[:])
X_train = df_train[df_train.columns[:-1]]
y_train = df_train["cnt"]

alphas_lasso = np.logspace(-7, -3, 500)
nb_folds = 10
reg = LassoCV(cv=nb_folds, alphas = alphas_lasso)
reg.fit(X, y)
print('Regularizator score (R2 coefficient):', reg.score(X, y))
print('Regularizator\'s optimal alpha value:', reg.alpha_)
print('Regularizator coefficients:')  
[(pair[0], round(pair[1], 4)) for pair in zip(df.columns[:-1], reg.coef_)]

We can plot the MSE as a function of $\alpha$ to check how much is gained in the cross validation process.

In [ ]:
mean_mse_arr = np.array(reg.mse_path_).mean(axis=1)
plt.figure(figsize=(10, 8)) 
plt.semilogx(reg.alphas_, mean_mse_arr, label=feature )
plt.xlabel("Alpha")
plt.ylabel("Mean MSE by fold")
plt.title("Lasso regularizator MSE")
#plt.ylim(0.011869,0.011872)
plt.show()

In [ ]:
# use cross validation for ridge regression 
# see https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.RidgeCV.html
from sklearn.linear_model import RidgeCV

In [ ]:
df_train, df_test = train_test_split(df, train_size = fraction_train, test_size = fraction_test)
df_train[:] = scaler.fit_transform(df_train[:])
X_train = df_train[df_train.columns[:-1]]
y_train = df_train["cnt"]

alphas_ridge = np.logspace(-5, 1, 500)
# doing "leave one out cross validation" by default 
reg = RidgeCV(cv=None, alphas = alphas_ridge, scoring=None, store_cv_values=True)
reg.fit(X, y)
print('Regularizator score (R2 coefficient):', reg.score(X, y))
print('Regularizator\'s optimal alpha value:', reg.alpha_)
print('Regularizator coefficients:')  
[(pair[0], round(pair[1], 4)) for pair in zip(df.columns[:-1], reg.coef_)]

In [ ]:
mean_mse_arr = np.array(reg.cv_values_).mean(axis=0)
plt.figure(figsize=(10, 8)) 
plt.semilogx(alphas_ridge, mean_mse_arr, label=feature )
plt.xlabel("Alpha")
plt.ylabel("Mean MSE by fold")
plt.title("Ridge regularizator MSE")
plt.show()

**Question 9.** What do you conclude?

We are not gaining much with lasso and ridge regression. The coefficients found by the two regularization strategies are rather similar. However, the accuracy is not great. It seems that the model is not able to make good predictions. More complex models should be tested.